# Exercise: SIMD Data Dependency

Consider the following loop involving four vectors `a`,`b`,`c`, and `d`:

In [1]:
const LOOP_ITERATIONS = 8192
const N = LOOP_ITERATIONS + 2

"naive loop"
function loop_naive!(a, b, c, d)
    @inbounds for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
        b[i+2] = c[i] + d[i]
    end
end

a = rand(Float32, N)
b = rand(Float32, N)
c = rand(Float32, N)
d = rand(Float32, N)

loop_naive!(a,b,c,d)

This loop is hard to auto-vectorize because it has a **data-dependency**: we're reading and writing elements of the vector `b`.

**Task 1**: Check the native code produced for `loop_naive!(a,b,c,d)` and convince yourself that the Julia compiler hasn't vectorized this code. (There shouldn't be any usage of `ymm` or `zmm` registers etc.)

In [2]:
@code_native debuginfo=:none syntax=:intel loop_naive!(a,b,c,d)

	.text
	.file	"loop_naive!"
	.globl	"japi1_loop_naive!_763"         # -- Begin function japi1_loop_naive!_763
	.p2align	4, 0x90
	.type	"japi1_loop_naive!_763",@function
"japi1_loop_naive!_763":                # @"japi1_loop_naive!_763"
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	mov	qword ptr [rbp - 56], rsi
	mov	rcx, qword ptr [rsi + 8]
	mov	rax, qword ptr [rsi]
	mov	rdx, qword ptr [rsi + 16]
	mov	rsi, qword ptr [rsi + 24]
	mov	rcx, qword ptr [rcx]
	mov	rax, qword ptr [rax]
	mov	rdx, qword ptr [rdx]
	mov	rsi, qword ptr [rsi]
	lea	rbx, [rcx + 32776]
	lea	rdi, [rax + 32768]
	lea	r8, [rdx + 32768]
	lea	r10, [rsi + 32768]
	cmp	rax, rbx
	setb	r13b
	cmp	rcx, rdi
	setb	r14b
	cmp	rax, r8
	setb	r11b
	cmp	rdx, rdi
	setb	r12b
	cmp	rax, r10
	setb	r9b
	cmp	rsi, rdi
	setb	r15b
	cmp	rcx, r8
	setb	r8b
	cmp	rdx, rbx
	setb	byte ptr [rbp - 41]             # 1-byte Folded Spill
	cmp	rcx, r10
	setb	dil
	cmp	rsi, rbx
	setb	r10b
	


**Task 2**: Implement the same loop in `loop_naive_simd!` and try to force SIMD-vectorization with the corresponding performance macro. (You shall keep the `@inbounds` as well.)

In [9]:
"naive loop + try force SIMD"
function loop_naive_simd!(a, b, c, d)
    #
    # TODO
    #
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
        b[i+2] = c[i] + d[i]
    end
end

loop_naive_simd!

**Task 3**: Check the native code of `loop_naive_simd!`. Has the code improved? The learning here is that just putting `@simd` in front of a loop and hoping for the best isn't a particularly good strategy 😉

In [10]:
@code_native debuginfo=:none syntax=:intel loop_naive_simd!(a,b,c,d)

	.text
	.file	"loop_naive_simd!"
	.globl	"japi1_loop_naive_simd!_1038"   # -- Begin function japi1_loop_naive_simd!_1038
	.p2align	4, 0x90
	.type	"japi1_loop_naive_simd!_1038",@function
"japi1_loop_naive_simd!_1038":          # @"japi1_loop_naive_simd!_1038"
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	mov	qword ptr [rbp - 56], rsi
	mov	rcx, qword ptr [rsi + 8]
	mov	rax, qword ptr [rsi]
	mov	rdx, qword ptr [rsi + 16]
	mov	rsi, qword ptr [rsi + 24]
	mov	rcx, qword ptr [rcx]
	mov	rax, qword ptr [rax]
	mov	rdx, qword ptr [rdx]
	mov	rsi, qword ptr [rsi]
	lea	rbx, [rcx + 32776]
	lea	rdi, [rax + 32768]
	lea	r8, [rdx + 32768]
	lea	r10, [rsi + 32768]
	cmp	rax, rbx
	setb	r13b
	cmp	rcx, rdi
	setb	r14b
	cmp	rax, r8
	setb	r11b
	cmp	rdx, rdi
	setb	r12b
	cmp	rax, r10
	setb	r9b
	cmp	rsi, rdi
	setb	r15b
	cmp	rcx, r8
	setb	r8b
	cmp	rdx, rbx
	setb	byte ptr [rbp - 41]             # 1-byte Folded Spill
	cmp	rcx, r10
	setb	dil
	cm

**Task 4**: Benchmark and compare the variants. What do you observe?


In [5]:
using BenchmarkTools

walltime = @belapsed loop_naive!($a, $b, $c, $d) samples = 5 evals = 3
println("the naive loop: ", round(walltime * 1e6; digits=2), " μs")
walltime = @belapsed loop_naive_simd!($a, $b, $c, $d) samples = 5 evals = 3
println("the naive loop + `@simd`: ", round(walltime * 1e6; digits=2), " μs")

the naive loop: 2.62 μs
the naive loop + `@simd`: 2.63 μs



**Task 5**: Take a closer look at the loop. Can you "resolve" the data-dependency issue by splitting up the loop into two separate loops? Implement this improved version in the functions below. Use `@simd` for the loops in the second function. (Again, keep `@inbounds` for all loops in both functions.)

In [6]:
"optimized loop"
function loop_opt!(a, b, c, d)
    #
    # TODO
    #
    @inbounds for i in 1:LOOP_ITERATIONS
        b[i+2] = c[i] + d[i]
    end
    @inbounds for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
    end
end

"optimized loop + `@simd`"
function loop_opt_simd!(a, b, c, d)
    #
    # TODO
    #
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        b[i+2] = c[i] + d[i]
    end
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
    end
end

loop_opt_simd!

**Task 6**: Benchmark those new variants as well.
  * How do they compare to each other?
  * Did the SIMD performance macro help? (Hint: It shouldn't.)
  * How does the performance compare to the unoptimized variants above?

In [7]:
walltime = @belapsed loop_opt!($a, $b, $c, $d) samples = 5 evals = 3
println("the optimized loop: ", round(walltime * 1e6; digits=2), " μs")
walltime = @belapsed loop_opt_simd!($a, $b, $c, $d) samples = 5 evals = 3
println("the optimized loop + `@simd`: ", round(walltime * 1e6; digits=2), " μs")

the optimized loop: 1.67 μs
the optimized loop + `@simd`: 1.66 μs



**Task 7**: Check the native code of e.g. `loop_opt_simd!`. Did it vectorize properly? (Look e.g. for `ymm` and `zmm` registers as well as a block of `vaddps` instructions. Note though, that this is system-dependent.)

In [8]:
@code_native debuginfo=:none syntax=:intel loop_opt_simd!(a, b, c, d)

	.text
	.file	"loop_opt_simd!"
	.globl	"japi1_loop_opt_simd!_1036"     # -- Begin function japi1_loop_opt_simd!_1036
	.p2align	4, 0x90
	.type	"japi1_loop_opt_simd!_1036",@function
"japi1_loop_opt_simd!_1036":            # @"japi1_loop_opt_simd!_1036"
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	mov	qword ptr [rbp - 8], rsi
	mov	rax, qword ptr [rsi + 8]
	mov	rcx, qword ptr [rsi + 16]
	mov	r8, qword ptr [rsi]
	mov	rsi, qword ptr [rsi + 24]
	mov	rdx, qword ptr [rcx]
	mov	rax, qword ptr [rax]
	mov	rsi, qword ptr [rsi]
	lea	rcx, [rax + 8]
	lea	r9, [rdx + 32768]
	lea	rdi, [rax + 32776]
	lea	r10, [rsi + 32768]
	cmp	rcx, r9
	setb	r9b
	cmp	rdx, rdi
	setb	r11b
	cmp	rcx, r10
	setb	cl
	cmp	rsi, rdi
	setb	dil
	test	r9b, r11b
	jne	.LBB0_8
# %bb.1:                                # %top
	and	cl, dil
	jne	.LBB0_8
# %bb.2:                                # %vector.body.preheader
	lea	rdi, [rax + 2024]
	add	rdx, 2016
	add	rsi, 2016
	xor	ecx, ecx
	.p2align	4, 0x90
.LBB0_3:       